In [1]:
import tkinter as tk
import copy
from tkinter import messagebox
import random

In [2]:
WIDTH, HEIGHT = 400, 400
CELL_SIZE = 10
GRID_WIDTH = WIDTH // CELL_SIZE
GRID_HEIGHT = HEIGHT // CELL_SIZE

In [3]:
grid = [[0 for _ in range(GRID_WIDTH)] for _ in range(GRID_HEIGHT)]
game_started = False
prev_grid = copy.deepcopy(grid)
prev_prev_grid = copy.deepcopy(grid)

In [4]:
def draw_grid():
    canvas.delete("all")
    for y in range(GRID_HEIGHT):
        for x in range(GRID_WIDTH):
            if grid[y][x]:
                canvas.create_rectangle(
                    x * CELL_SIZE, y * CELL_SIZE,
                    (x + 1) * CELL_SIZE, (y + 1) * CELL_SIZE,
                    fill="green"
                )
generation_count = 0

In [5]:
def update_generation_count_label():
    global generation_count
    generation_count += 1
    generation_label.config(text=f"Поколение: {generation_count}")

In [6]:
def toggle_cell(event):
    x = event.x // CELL_SIZE
    y = event.y // CELL_SIZE
    if 0 <= x < GRID_WIDTH and 0 <= y < GRID_HEIGHT:
        grid[y][x] = 1 - grid[y][x]
        draw_grid()

In [7]:
def place_glider():
    if not game_started:
        clear_grid()
        glider = [
            [0, 1, 0],
            [0, 0, 1],
            [1, 1, 1]
        ]
        for y in range(len(glider)):
            for x in range(len(glider[0])):
                grid[y][x] = glider[y][x]
        draw_grid()

In [8]:
def start_game():
    global game_started, prev_grid, prev_prev_grid
    game_started = True
    start_button.config(state="disabled")
    stop_button.config(state="normal")
    clear_button.config(state="disabled")
    prev_grid = copy.deepcopy(grid)
    prev_prev_grid = copy.deepcopy(grid)
    step()

In [9]:
def stop_game():
    global game_started
    game_started = False
    start_button.config(state="normal")
    stop_button.config(state="disabled")
    clear_button.config(state="normal")
    if check_cells_dead():
        messagebox.showinfo("Игра завершена", "Игра завершена. Все клетки умерли.")
    elif is_cycle_detected():
        messagebox.showinfo("Игра завершена", "Игра завершена. Клетки зациклились.")

In [10]:
def clear_grid():
    global grid, generation_count
    if not game_started:
        grid = [[0 for _ in range(GRID_WIDTH)] for _ in range(GRID_HEIGHT)]
        draw_grid()
        generation_count = 0
        generation_label.config(text=f"Поколение: {generation_count}")


In [11]:
def randomize_grid():
    global grid
    if not game_started:
        grid = [[random.randint(0, 1) for _ in range(GRID_WIDTH)] for _ in range(GRID_HEIGHT)]
        draw_grid()

In [12]:
def is_game_over():
    return grid == prev_grid

In [13]:
def is_cycle_detected():
    return grid == prev_prev_grid

In [14]:
def check_cells_dead():
    return not any(cell for row in grid for cell in row)

In [15]:
def step():
    global grid, prev_grid, prev_prev_grid
    if game_started:
        new_grid = [[0 for _ in range(GRID_WIDTH)] for _ in range(GRID_HEIGHT)]
        for y in range(GRID_HEIGHT):
            for x in range(GRID_WIDTH):
                neighbors = 0
                for dx in [-1, 0, 1]:
                    for dy in [-1, 0, 1]:
                        if dx == 0 and dy == 0:
                            continue
                        nx, ny = x + dx, y + dy
                        if 0 <= nx < GRID_WIDTH and 0 <= ny < GRID_HEIGHT and grid[ny][nx]:
                            neighbors += 1
                if grid[y][x] == 1 and (neighbors < 2 or neighbors > 3):
                    new_grid[y][x] = 0
                elif grid[y][x] == 0 and neighbors == 3:
                    new_grid[y][x] = 1
                else:
                    new_grid[y][x] = grid[y][x]
        prev_prev_grid = copy.deepcopy(prev_grid)
        prev_grid = copy.deepcopy(grid)
        grid = new_grid
        draw_grid()
        update_generation_count_label()
        if is_cycle_detected() or check_cells_dead():
            stop_game()
        else:
            root.after(100, step)

In [16]:
def show_about_info():
    about_info = """
    Игра "Жизнь" на Python с использованием Tkinter.
    Автор: Алибеков Аслан
    Год: 2023
    """
    messagebox.showinfo("О программе", about_info)

In [17]:
def show_placement_buttons():
    placement_window = tk.Toplevel(root)
    placement_window.title("Выберите размещение")

    glider_button = tk.Button(placement_window, text="Глайдер", command=place_glider, font=("Helvetica", 10))
    glider_button.pack()

In [18]:
root = tk.Tk()
root.title("Игра 'Жизнь'")
root.geometry("500x500")

menu = tk.Menu(root)
root.config(menu=menu)

help_menu = tk.Menu(menu, tearoff=0)
menu.add_cascade(label="Справка", menu=help_menu)
help_menu.add_command(label="О программе", command=show_about_info)

frame = tk.Frame(root)
frame.pack()

canvas = tk.Canvas(frame, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

canvas.bind("<Button-1>", toggle_cell)

button_frame = tk.Frame(root)
button_frame.pack()

start_button = tk.Button(button_frame, text="Старт", command=start_game, font=("Helvetica", 10))
start_button.pack(side="left")

stop_button = tk.Button(button_frame, text="Стоп", command=stop_game, state="disabled", font=("Helvetica", 10))
stop_button.pack(side="left")

clear_button = tk.Button(button_frame, text="Очистить поле", command=clear_grid, font=("Helvetica", 10))
clear_button.pack(side="left")

random_button = tk.Button(button_frame, text="Случайное заполнение", command=randomize_grid, font=("Helvetica", 10))
random_button.pack(side="left")

placement_menu_button = tk.Button(button_frame, text="Размещение", command=show_placement_buttons, font=("Helvetica", 10))
placement_menu_button.pack(side="left")

generation_label = tk.Label(root, text="Поколение: 0", font=("Helvetica", 10))
generation_label.pack()

button_frame.place(relx=0.5, rely=0.9, anchor="s")

root.mainloop()